<div class="alert alert-success">  
    
-------
# XArray 101 🌍  
-------
* Jupyter and Python Basics
* Xarray Intro
* __Xarray Advanced__
* Vector Data
* Remote Sensing
* Visualization

-------  
</div>

# Some more XArray Analysis

Example from: https://rabernat.github.io/research_computing_2018/intermediate-xarray.html  

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
import xarray as xr
import numpy as np
import matplotlib.pylab as plt

First, we want to replicate this plot here... It shows the SST ENSO anomalies...

Lets try to reproduce this...

![](https://www.ncdc.noaa.gov/monitoring-content/teleconnections/eln-f-pg.gif)

In [ ]:
ds = xr.open_dataset('NOAA_NCDC_ERSST_v3b_SST.nc')
ds

In [ ]:
# convert to -180/180 
ds.coords['lon'] = (ds.coords['lon'] + 180) % 360 - 180
ds = ds.sortby(ds.lon)

Let's select the regions first.

In [ ]:
a = ds.sel(lat=slice(-5,5), lon=slice(-170,-120)).mean(dim=['lat','lon'])
a

In [ ]:
a.sst.plot();

Then we compute the diff between value and average value for the sst variable. We also do a rolling window smooth (size=3)...

In [ ]:
ts = (a - a.mean()).sst.rolling(time=3, center=True).mean()
ts.plot();

And now we try to replicate the plot by going into details with matplotlib...

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(8,6))
y2=np.zeros_like(ts.values)

ax.hlines(0.5, ts.time.values[0], ts.time.values[-1], color='red', ls='--')
ax.hlines(-0.5, ts.time.values[0], ts.time.values[-1], color='blue', ls='--')
ax.plot(ts.time.values, ts.values, c='black')
ax.fill_between(ts.time.values, ts.values, y2, where=ts.values > 0, facecolor='orange', lw=.5)
ax.fill_between(ts.time.values, ts.values, y2, where=ts.values <=0, facecolor='cyan', lw=.5)
ax.set_xlim(ts.time.values[0], ts.time.values[-1])
ax.set_ylim(-3,3)
ax.set_ylabel('Anomaly in Degrees C')
ax.set_xlabel('Year');